In [ ]:
import numpy as np

from models import *
from utils import *

import os, sys, time, datetime, random
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch.autograd import Variable

import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
import cv2
import glob
import pandas as pd

In [47]:
config_path='config/yolov3.cfg'
weights_path='config/yolov3.weights'
class_path='config/coco.names'
img_size=416
conf_thres=0.9
nms_thres=0.7

# Load model and weights
model = Darknet(config_path, img_size=img_size)
model.load_weights(weights_path)
model.cuda()
model.eval()
classes = utils.load_classes(class_path)
Tensor = torch.cuda.FloatTensor
from skimage.transform import resize

C:\Users\Mechano_2\anaconda3\envs\yolov3_pytorch_on_backside_crack\lib\site-packages\torch\nn\_reduction.py:44: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


In [48]:
def detect_image(img):
    # # scale and pad image
    # # img = transforms.ToPILImage()(img)
    # ratio = min(img_size/img.shape[0], img_size/img.shape[1])
    # imw = round(img.shape[1] * ratio)
    # imh = round(img.shape[0] * ratio)
    # print(ratio)
    # print(imw)
    # print(imh)
    # img = cv2.resize(img, (imw,imh))
    # print(img.shape)
    # plt.imshow(img)
    # left= max(int((imh-imw)/2),0)
    # right= max(int((imh-imw)/2),0)
    # top = max(int((imw-imh)/2),0)
    # bottom = max(int((imw-imh)/2),0)
    # img = np.pad(img, ((top, bottom),(left,right),(0,0)), mode = 'constant')
    # img = np.array(img, dtype='float')
    # print(np.amax(img[:,:,0]))
    # print(img.shape)
    # print(img[300][300])
    # # img = img/255.0
    # # print(img[200][200])
    # print(img[:,:,0][300][300]/255.0)
    # img[:,:,0]=(img[:,:,0]-np.amin(img[:,:,0]))/(np.amax(img[:,:,0])-np.amin(img[:,:,0]))
    # img[:,:,1]=(img[:,:,1]-np.amin(img[:,:,1]))/(np.amax(img[:,:,1])-np.amin(img[:,:,1]))
    # img[:,:,2]=(img[:,:,2]-np.amin(img[:,:,2]))/(np.amax(img[:,:,2])-np.amin(img[:,:,2]))
    # # img[:,:,0]=(img[:,:,0])/255.0
    # # img[:,:,1]=(img[:,:,1])/255.0
    # # img[:,:,2]=(img[:,:,2])/255.0
    # print(img.shape)
    # print(img[300][300])
    # print(np.amax(img[:,:,0]))
    #
    #
    # # print(img1[300][300][0])
    # # img[:,:,0]=(img[:,:,0]-np.amin(img[:,:,0]))/(np.amax(img[:,:,0])-np.amin(img[:,:,0]))
    # # img[:,:,1]=(img[:,:,1]-np.amin(img[:,:,1]))/(np.amax(img[:,:,1])-np.amin(img[:,:,1]))
    # # img[:,:,2]=(img[:,:,2]-np.amin(img[:,:,2]))/(np.amax(img[:,:,2])-np.amin(img[:,:,2]))
    #
    # # img[:,:,1]=(img[:,:,1]-np.min(img[:,:,1]))/(np.max(img[:,:,1])-np.min(img[:,:,1]))
    # # img[:,:,2]=(img[:,:,2]-np.min(img[:,:,2]))/(np.max(img[:,:,2])-np.min(img[:,:,2]))
    # # trans = transforms.Compose([transforms.ToTensor()])
    # # image_tensor = trans(img).float()
    #
    # plt.imshow(img)
    #
    # # img = output = cv2.copyMakeBorder(img, max(int((imh-imw)/2),0), max(int((imh-imw)/2),0), max(int((imw-imh)/2),0), max(int((imw-imh)/2),0), cv2.BORDER_CONSTANT, (128,128,128))
    #
    # img = np.moveaxis(img,-1,0)
    # # print(img.shape)
    # # img = np.expand_dims(img,axis=0)
    # # print(img.shape)
    # # img_transforms = transforms.Compose([ transforms.Resize((imh, imw)),
    # #      transforms.Pad((max(int((imh-imw)/2),0), max(int((imw-imh)/2),0), max(int((imh-imw)/2),0), max(int((imw-imh)/2),0)),
    # #                     (128,128,128)),
    # #      transforms.ToTensor(),
    # #      ])
    # # convert image to Tensor
    # img = torch.from_numpy(img)
    # # print(img.shape)
    # #
    # image_tensor = img.float()
    #
    # image_tensor = image_tensor.unsqueeze_(0)
    # print(image_tensor.shape)
    # input_img = Variable(image_tensor.type(Tensor))
    # print(input_img[0][0][213][213])
    img[:, :, 0] = 2 * (img[:, :, 0] - np.amin(img[:, :, 0])) / (np.amax(img[:, :, 0]) - np.amin(img[:, :, 0])) - 1
    img[:, :, 1] = 2 * (img[:, :, 1] - np.amin(img[:, :, 1])) / (np.amax(img[:, :, 1]) - np.amin(img[:, :, 1])) - 1
    img[:, :, 2] = 2 * (img[:, :, 2] - np.amin(img[:, :, 2])) / (np.amax(img[:, :, 2]) - np.amin(img[:, :, 2])) - 1
    h, w, _ = img.shape
    dim_diff = np.abs(h - w)
        # Upper (left) and lower (right) padding
    pad1, pad2 = dim_diff // 2, dim_diff - dim_diff // 2
        # Determine padding
    pad = ((pad1, pad2), (0, 0), (0, 0)) if h <= w else ((0, 0), (pad1, pad2), (0, 0))
    img = np.pad(img, pad, 'constant', constant_values=0.0)
        # Add padding
    # img[:, :, 0] = (img[:, :, 0] - np.amin(img[:, :, 0])) / (np.amax(img[:, :, 0]) - np.amin(img[:, :, 0]))
    # img[:, :, 1] = (img[:, :, 1] - np.amin(img[:, :, 1])) / (np.amax(img[:, :, 1]) - np.amin(img[:, :, 1]))
    # img[:, :, 2] = (img[:, :, 2] - np.amin(img[:, :, 2])) / (np.amax(img[:, :, 2]) - np.amin(img[:, :, 2]))
    # img[:, :, 0] = 2 * (img[:, :, 0] - np.amin(img[:, :, 0])) / (np.amax(img[:, :, 0]) - np.amin(img[:, :, 0])) - 1
    # img[:, :, 1] = 2 * (img[:, :, 1] - np.amin(img[:, :, 1])) / (np.amax(img[:, :, 1]) - np.amin(img[:, :, 1])) - 1
    # img[:, :, 2] = 2 * (img[:, :, 2] - np.amin(img[:, :, 2])) / (np.amax(img[:, :, 2]) - np.amin(img[:, :, 2])) - 1
    # input_img = np.pad(img, pad, 'constant', constant_values=127.5) / 255.
    # input_img = np.pad(img, pad, 'constant', constant_values=0.0)
        # Resize and normalize
    input_img = resize(img, (img_size, img_size, 3), mode='reflect')

        # Channels-first
    input_img = np.transpose(input_img, (2, 0, 1))
        # As pytorch tensor
    image_tensor = torch.from_numpy(input_img).float()
    image_tensor = image_tensor.unsqueeze_(0)
    input_img = Variable(image_tensor.type(Tensor))

    # run inference on the model and get detections
    with torch.no_grad():
        detections = model(input_img)
        detections = utils.non_max_suppression(detections, 6, conf_thres, nms_thres)
    return detections[0]

In [49]:
# load image and get detections
img_path = list(glob.glob('D:/PycharmProjects/Hiddencrack/tf-keras-frcnn/random_cropped_15_lang/images/circle_2p5/*.npy'))
print(img_path)

['D:/PycharmProjects/Hiddencrack/tf-keras-frcnn/random_cropped_15_lang/images/circle_2p5\\000circle_2p5_0.npy', 'D:/PycharmProjects/Hiddencrack/tf-keras-frcnn/random_cropped_15_lang/images/circle_2p5\\000circle_2p5_1.npy', 'D:/PycharmProjects/Hiddencrack/tf-keras-frcnn/random_cropped_15_lang/images/circle_2p5\\001circle_2p5_0.npy', 'D:/PycharmProjects/Hiddencrack/tf-keras-frcnn/random_cropped_15_lang/images/circle_2p5\\001circle_2p5_1.npy', 'D:/PycharmProjects/Hiddencrack/tf-keras-frcnn/random_cropped_15_lang/images/circle_2p5\\002circle_2p5_0.npy', 'D:/PycharmProjects/Hiddencrack/tf-keras-frcnn/random_cropped_15_lang/images/circle_2p5\\002circle_2p5_1.npy', 'D:/PycharmProjects/Hiddencrack/tf-keras-frcnn/random_cropped_15_lang/images/circle_2p5\\003circle_2p5_0.npy', 'D:/PycharmProjects/Hiddencrack/tf-keras-frcnn/random_cropped_15_lang/images/circle_2p5\\003circle_2p5_1.npy', 'D:/PycharmProjects/Hiddencrack/tf-keras-frcnn/random_cropped_15_lang/images/circle_2p5\\004circle_2p5_0.npy', 

In [50]:
for i in range(len(img_path)):
# img_path = "images/blueangels.npy"
    frame=os.path.splitext(os.path.basename(img_path[i]))[0]
    print(frame)
    prev_time = time.time()
# img = Image.open(img_path)
#     img = np.load(img_path[i])
    img = np.array(np.load(img_path[i]), dtype='float')
    # print(img[300][600])
    print(img.shape)
    detections = detect_image(img)
    inference_time = datetime.timedelta(seconds=time.time() - prev_time)
    print ('Inference Time: %s' % (inference_time))

# Get bounding-box colors
    cmap = plt.get_cmap('tab20b')
    colors = [cmap(i) for i in np.linspace(0, 1, 20)]

    img = np.array(img)
    # plt.figure()
    # fig, ax = plt.subplots(1, figsize=(5,5))
    # shw = ax.contourf(img[:, :, 0], 20, cmap='jet')
    # bar = plt.colorbar(shw)
    # ax.invert_yaxis()
    # ax.imshow(img)

    pad_x = max(img.shape[0] - img.shape[1], 0) * (img_size / max(img.shape))
    pad_y = max(img.shape[1] - img.shape[0], 0) * (img_size / max(img.shape))
    unpad_h = img_size - pad_y
    unpad_w = img_size - pad_x
    print(unpad_w)
    print(unpad_h)
    if detections is not None:
       unique_labels = detections[:, -1].cpu().unique()
       n_cls_preds = len(unique_labels)
       bbox_colors = random.sample(colors, n_cls_preds)
    # browse detections and draw bounding boxes
       for x1, y1, x2, y2, conf, cls_conf, cls_pred in detections:
           box_h = (((y2 - y1) / unpad_h) * img.shape[0])
           box_w = ((x2 - x1) / unpad_w) * img.shape[1]
           y1 = ((y1 - pad_y // 2) / unpad_h) * img.shape[0]
           x1 = ((x1 - pad_x // 2) / unpad_w) * img.shape[1]
           conf = "{:.2f}".format(conf*100)
           cls_conf = "{:.2f}".format(cls_conf*100)
           record_df = pd.DataFrame(columns=['frame', 'probability', 'key','xmin','ymin','w','h'])
           new_row = {'frame':frame,'probability' : cls_conf,'key': classes[int(cls_pred.cpu())],'xmin':x1.cpu().numpy(),'ymin':y1.cpu().numpy(),'w':box_w.cpu().numpy(),'h':box_h.cpu().numpy()}
           record_df = record_df.append(new_row, ignore_index=True)
           record_df.to_csv('data_file.csv',mode='a', index=False,header=not os.path.exists('data_file.csv'))
           # print(conf)
           # print(cls_conf)
           # print(y1)
           # print(x1)
           # print()
           # color = bbox_colors[int(np.where(unique_labels == int(cls_pred))[0])]
           # bbox = patches.Rectangle((x1.cpu(), y1.cpu()), box_w.cpu(), box_h.cpu(), linewidth=2, edgecolor=color, facecolor='none')
           # ax.add_patch(bbox)
           # plt.text(x1, y1, s='{}:{}'.format(classes[int(cls_pred)],cls_conf),fontsize=20, color='white', verticalalignment='top',
           #      bbox={'color': color, 'pad': 0})
           # plt.axis('off')
# save image
#     plt.savefig('D:/yolov3_pytorch_on_backside_crack/save_result_png/'+str(frame)+'.png', bbox_inches='tight', pad_inches=0.0)
    # plt.show()

000circle_2p5_0
(600, 400, 3)
Inference Time: 0:00:00.385121
277.3333333333333
416.0
000circle_2p5_1
(600, 400, 3)


C:\Users\Mechano_2\AppData\Local\Temp\ipykernel_28832\1081909300.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  record_df = record_df.append(new_row, ignore_index=True)


Inference Time: 0:00:00.221772
277.3333333333333
416.0
001circle_2p5_0
(600, 400, 3)
Inference Time: 0:00:00.224434
277.3333333333333
416.0
001circle_2p5_1
(600, 400, 3)


C:\Users\Mechano_2\AppData\Local\Temp\ipykernel_28832\1081909300.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  record_df = record_df.append(new_row, ignore_index=True)
C:\Users\Mechano_2\AppData\Local\Temp\ipykernel_28832\1081909300.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  record_df = record_df.append(new_row, ignore_index=True)


Inference Time: 0:00:00.225687
277.3333333333333
416.0
002circle_2p5_0
(600, 400, 3)


...

(600, 400, 3)
Inference Time: 0:00:00.186929
277.3333333333333
416.0
073circle_2p5_0
(600, 400, 3)
Inference Time: 0:00:00.178483
277.3333333333333
416.0
073circle_2p5_1
(600, 400, 3)
Inference Time: 0:00:00.190480
277.3333333333333
416.0
074circle_2p5_0
(600, 400, 3)
Inference Time: 0:00:00.260521
277.3333333333333
416.0
074circle_2p5_1
(600, 400, 3)
Inference Time: 0:00:00.147013
277.3333333333333
416.0
075circle_2p5_0
(600, 400, 3)
Inference Time: 0:00:00.224007
277.3333333333333
416.0
075circle_2p5_1
(600, 400, 3)
Inference Time: 0:00:00.192762
277.3333333333333
416.0


C:\Users\Mechano_2\AppData\Local\Temp\ipykernel_28832\1081909300.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  record_df = record_df.append(new_row, ignore_index=True)
C:\Users\Mechano_2\AppData\Local\Temp\ipykernel_28832\1081909300.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  record_df = record_df.append(new_row, ignore_index=True)


076circle_2p5_0
(600, 400, 3)
Inference Time: 0:00:00.201508
277.3333333333333
416.0
076circle_2p5_1
(600, 400, 3)
Inference Time: 0:00:00.201185
277.3333333333333
416.0
077circle_2p5_0
(600, 400, 3)
Inference Time: 0:00:00.223779
277.3333333333333
416.0
077circle_2p5_1
(600, 400, 3)
Inference Time: 0:00:00.202935
277.3333333333333
416.0
078circle_2p5_0
(600, 400, 3)
Inference Time: 0:00:00.185941
277.3333333333333
416.0
078circle_2p5_1
(600, 400, 3)
Inference Time: 0:00:00.195289
277.3333333333333
416.0
079circle_2p5_0
(600, 400, 3)


C:\Users\Mechano_2\AppData\Local\Temp\ipykernel_28832\1081909300.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  record_df = record_df.append(new_row, ignore_index=True)


Inference Time: 0:00:00.185793
277.3333333333333
416.0
079circle_2p5_1
(600, 400, 3)
Inference Time: 0:00:00.190696
277.3333333333333
416.0
080circle_2p5_0
(600, 400, 3)
Inference Time: 0:00:00.189939
277.3333333333333
416.0
080circle_2p5_1
(600, 400, 3)
Inference Time: 0:00:00.205934
277.3333333333333
416.0
081circle_2p5_0
(600, 400, 3)
Inference Time: 0:00:00.193959
277.3333333333333
416.0
081circle_2p5_1
(600, 400, 3)
Inference Time: 0:00:00.192314
277.3333333333333
416.0
082circle_2p5_0
(600, 400, 3)
Inference Time: 0:00:00.278601
277.3333333333333
416.0
082circle_2p5_1
(600, 400, 3)
Inference Time: 0:00:00.222203
277.3333333333333
416.0
083circle_2p5_0
(600, 400, 3)
Inference Time: 0:00:00.182066
277.3333333333333
416.0
083circle_2p5_1
(600, 400, 3)
Inference Time: 0:00:00.273839
277.3333333333333
416.0
084circle_2p5_0
(600, 400, 3)
Inference Time: 0:00:00.138711
277.3333333333333
416.0
084circle_2p5_1
(600, 400, 3)
Inference Time: 0:00:00.165117
277.3333333333333
416.0
085circle_